# Analysis of Contours of the HCP

## Configuration

In [1]:
from pathlib import Path
# The cache directories:
cache_path       = Path('/data/crcns2021/hcpannot-cache')
image_cache_path = cache_path / 'annot-images'
v123_cache_path  = cache_path / 'annot-v123'
csulc_cache_path = cache_path / 'annot-csulc'
# The save path of the data branch:
data_path = Path('/data/crcns2021/results/data_branch/save')
# The processing path and subpaths:
proc_path    = Path('/data/crcns2021/results/proc')
traces_path  = proc_path / 'traces'
paths_path   = proc_path / 'paths'
means_path   = proc_path / 'means'
labels_path  = proc_path / 'labels'
reports_path = proc_path / 'reports'

# The file of visual surface areas for the data.
ventral_sarea_path = proc_path / 'ventral_sareas.tsv'
dorsal_sarea_path = proc_path / 'dorsal_sareas.tsv'

# The hcpannot library path; if hcpannot is not on the path for
# this notebook, the notebook will try to figure out where it is
# and will use this directory as a backup.
import sys
sys.path.append('..')

# If you aren't using /data
import os
os.environ['HCPANNOT_LOAD_PATH'] = os.fspath(cache_path)

## Meta-data about the Annotations

In [2]:
ventral_raters = [
    'R1', #'BrendaQiu',
    'R2', #'JiyeongHa',
    'R3', #'bogengsong',
    'R4', #'lindazelinzhao',
    'R5'] #'nourahboujaber'
dorsal_raters = [
    'R6', #'Annie-lsc',
    'R1', #'BrendaQiu',
    'R7', #'mominbashir',
    'R8'] #'qiutan6li'
region_raters = {
    'ventral': ventral_raters,
    'dorsal': dorsal_raters}

# The hemispheres we are processing over.
hemis = ['lh', 'rh']

## Dependencies

In [3]:
import os, sys, pimms, pandas, json
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipyvolume as ipv
import torch
import neuropythy as ny

import hcpannot

# Note the cache path we want to use outside the docker container;
# normally this gets set by the Docker startup, so here outside
# the docker container we set it manually.
hcpannot.interface.default_load_path = str(cache_path)

# The subject IDs we are processing over, as a numpy array.
sids = np.array(hcpannot.config.subject_list)

# The list of visual areas by region:
from hcpannot.config import region_areas

# A utility function for getting data from proc dictionaries.
from hcpannot import nestget

# The mean rater's name ('mean') and some plotting functions are also defined in
# the hcpannot.analysis subpackage.
from hcpannot.io import plot_contours

# Finally, the proc and meanproc functions, which give us the processed data.
from hcpannot.proc import proc, proc_meanrater, proc_meansub

### How to Examine the Processed Contour Data

The `proc` function can be used to query processed data. `proc` returns a (lazy) dictionary of the processed data. If the requested data have not been previously calculated and cached, the `proc` function will do so, saving cached results to (and loading them from) the directory provided via the `save_path` option. The `load_path` refers to the directory from which the contours should be loaded, i.e. the `save` directory of the [`data` branch of the `noahbenson/hcp-annot-vc` GitHub repository](https://github.com/noahbenson/hcp-annot-vc/tree/data).

#### Examples:

* Load (or prepare to calculate) the processed ventral contour data for rater `R1`, HCP subject `100610`, and the left hemisphere.  
  ```python
  >>> from hcpannot.proc import proc 
  >>> data = proc(
  ...     'ventral',
  ...     rater='R1',
  ...     sid=100610,
  ...     hemisphere='lh',
  ...     save_path=proc_path,
  ...     load_path=data_path)
  ```
* View the keys of the processed data dictionary.  
  ```python
  >>> sorted(data.keys())
  ['chirality',
   'contours',
   'cortex',
   'flatmap',
   'hemisphere',
   'io_options',
   'label_weights',
   'labelkey',
   'labels',
   'nested_data',
   'paths',
   'rater',
   'region',
   'reports',
   'save_path',
   'sid',
   'traces']
  ```
* View the contours. Each contour is a `2xN` matrix of the (`x`,`y`)-values of the points clicked by the rater. The coordinates of the points are registered to the flatmap on which they were drawn, which is given by `data['flatmap']`.  
  ```python
  >>> data['contours']
  {'hV4_VO1': array([[ 38.02148285,  32.13910222, ...],
                     [-44.74639194, -46.70718548, ...]]),
   'VO1_VO2': array([[ 39.00187962,  38.02148285, ...],
                     [-44.25525238, -47.68664107, ...]]),
   'hV4_outer': array([[-18.51013327, -19.33172823, ...],
                       [-14.4353661 , -18.76493634, ...]]),
   'VO_outer': array([[ 38.80127474,  43.90386347, ...],
                      [-43.03350935, -44.74639194, ...]])}
  ```
* Plot a contour on the flatmap.  
  ```python
  >>> import neuropythy as ny, matplotlib.pyplot as plt
  >>> color = 'prf_polar_angle'
  >>> mask = ('prf_variance_explained', 0.1, 1)
  >>> (fig,ax) = plt.subplots(1,1, figsize=(8,8))
  >>> ny.cortex_plot(data['flatmap'], color=color, mask=mask, axes=ax)
  >>> (x, y) = data['contours']['hV4_outer']
  >>> ax.plot(x, y, 'wo-')
  >>> plt.show()
  ```

Here's an example code-block using the `plot_contours` function to plot a set of boundaries along with the V1-V3 contours.

```python
>>> data = proc(
...     'ventral',
...     rater='R1',
...     sid=100610,
...     hemisphere='lh',
...     save_path=proc_path,
...     load_path=data_path)

# Make the flatmap plot and the boundaries.
>>> fig = plot_contours(dat, boundaries=True)

# Extract the pyplot axes that were used.
>>> ax = fig.axes[0]

# Grab the subject data, which includes the V1-V3 contours.
>>> sdat = hcpannot.interface.subject_data[(dat['sid'],dat['hemisphere'])]

# And plot all of these contours:
>>> for (x,y) in sdat['v123'].values():
...     ax.plot(x, y, 'w-', lw=0.25)
```